<a href="https://colab.research.google.com/github/Praveenmurali986/keras-tuner-tutorial/blob/main/my_notebook_on_dl_keras_tuner_aug_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras


In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

importing datasets of fashion from keras library

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
label_train[3]

In [ ]:
img_train[3].shape

In [ ]:
img_train[3]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_train[0], cmap='binary')
plt.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_train[1], cmap='binary')
plt.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_train[2], cmap='binary')
plt.axis('off')
plt.show()

In [ ]:
#normalizing
img_train=img_train.astype('float32')/255
img_test=img_test.astype('float32')/255

In [ ]:
img_train[0]

In [ ]:
def model_builder(hp):
  model=keras.Sequential()
  #inputlayer
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  hp_units=hp.Int('units',min_value=32,max_value=512,step=32)

  model.add(keras.layers.Dense(units=hp_units,activation='relu'))
  model.add(keras.layers.Dense(10))#output layer

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001

  hp_learning_rate=hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    #crossentropy since the problem is multiclass.
  return model


In [ ]:
tuner=kt.Hyperband(model_builder,
                   objective='val_accuracy',
                   max_epochs=10,
                   factor=3,
                   directory='my_dir',
                   project_name='intro')

In [ ]:
early_stoping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[early_stoping])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


In [ ]:
#trainig model with the tuned values
model=tuner.hypermodel.build(best_hps)
history=model.fit(img_train,label_train,epochs=50 ,validation_split=0.2,callbacks=early_stoping)



In [ ]:
val_acc_per_epoch=history.history['val_accuracy']
best_epoch=val_acc_per_epoch.index(max(val_acc_per_epoch))+1
print('best epoch : %d'%(best_epoch,))

In [ ]:
model=tuner.hypermodel.build(best_hps)
history=model.fit(img_train,label_train,epochs=best_epoch ,validation_split=0.2)

In [ ]:
eval_result=model.evaluate(img_test,label_test)
print('[test loss, test accuracy]: ' , eval_result)